In [1]:
#!pip install -U transformers rank_bm25 lancedb unstructured --quiet

In [2]:
import transformers
transformers.__version__

'4.38.1'

In [229]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from model import EncoderModel, DecoderModel, BM25Model
from store import VectorStore
from tqdm import tqdm
import torch

# Dataset

In [4]:
ds = load_dataset("squad_v2")
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [5]:
df_train = ds["train"].to_pandas()[["context", "question", "answers"]]
display(df_train.head(3))
df_val = ds["validation"].to_pandas()[["context", "question", "answers"]]
display(df_val.head(3))

,context,question,answers
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start'..."
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"{'text': ['singing and dancing'], 'answer_star..."
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"{'text': ['2003'], 'answer_start': [526]}"


,context,question,answers
0,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'Franc..."
1,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 1..."
2,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"{'text': ['Denmark, Iceland and Norway', 'Denm..."


In [6]:
extract_answers = lambda answer: "" if len(answer['text']) == 0 else answer['text'][0]
v_extract_answers = np.vectorize(extract_answers)

In [7]:
df_train["answers"] = v_extract_answers(df_train["answers"].values)
df_val["answers"] = v_extract_answers(df_val["answers"].values)

print(f"{df_train[df_train['answers'] == ''].shape[0]}/{df_train.shape[0]}")
print(f"{df_val[df_val['answers'] == ''].shape[0]}/{df_val.shape[0]}")

43498/130319
5945/11873


In [13]:
df_val.sample(n=3, random_state=1)

,context,question,answers
7105,"Research by Harvard economist Robert Barro, fo...",What does low levels of inequality do for econ...,
10938,The FSO Car Factory was established in 1951. A...,Who bought the factory in 2005?,AvtoZAZ
11450,British victories continued in all theaters in...,What battle outside Quebec City did British lo...,


In [230]:
db = VectorStore("sentence-transformers/all-MiniLM-L6-v2")

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [234]:
db.add_documents(df_train["context"].values[:1000].tolist())

In [237]:
db.search(df_train["question"].values[0])

[{'score': 0.5412642226321411,
  'document': 'In The New Yorker music critic Jody Rosen described Beyoncé as "the most important and compelling popular musician of the twenty-first century..... the result, the logical end point, of a century-plus of pop." When The Guardian named her Artist of the Decade, Llewyn-Smith wrote, "Why Beyoncé? [...] Because she made not one but two of the decade\'s greatest singles, with Crazy in Love and Single Ladies (Put a Ring on It), not to mention her hits with Destiny\'s Child; and this was the decade when singles – particularly R&B singles – regained their status as pop\'s favourite medium. [...] [She] and not any superannuated rock star was arguably the greatest live performer of the past 10 years." In 2013, Beyoncé made the Time 100 list, Baz Luhrmann writing "no one has that voice, no one moves the way she moves, no one can hold an audience the way she does... When Beyoncé does an album, when Beyoncé sings a song, when Beyoncé does anything, it\'s

In [12]:
# for bm_25 in [False, True]:
    # embed documents
    # for model in bert_models
        # for ... (batch)

        # retieve and answer
        # for model in causal_models
            # for distance metric in ...
                # ndcg
            
            # for query
                # BLEU, ROUGE, RAGAs, F1(?) 